# Kmeans Clustering

In [1]:
import time

import pyspark
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [2]:
# Change the number of cores in this code block
# by setting `spark.master` to `local[n]` where
# n is the number of cores

conf = pyspark.SparkConf().setAll([('spark.master', 'local[4]'),
                                   ('spark.app.name', 'Basic Setup')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-20 00:43:03,983 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("header",True).csv("file:///home/work/data/final_movies_df_10.csv", inferSchema=True)
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- movieYear: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- Crime: double (nullable = true)
 |-- Romance: double (nullable = true)
 |-- Thriller: double (nullable = true)
 |-- Adventure: double (nullable = true)
 |-- Drama: double (nullable = true)
 |-- War: double (nullable = true)
 |-- Documentary: double (nullable = true)
 |-- Fantasy: double (nullable = true)
 |-- Mystery: double (nullable = true)
 |-- Musical: double (nullable = true)
 |-- Animation: double (nullable = true)
 |-- Film-Noir: double (nullable = true)
 |-- Horror: double (nullable = true)
 |-- Western: double (nullable = true)
 |-- Comedy: double (nullable = true)
 |-- Children: double (nullable = true)
 |-- Action: double (nullable = true)
 |-- Sci-Fi: double (nullable = true)



In [4]:
df.show(5)

+-------+--------------------+------------------+---------+--------------------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+
|movieId|               title|        avg_rating|movieYear|              genres|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|Horror|Western|Comedy|Children|Action|Sci-Fi|
+-------+--------------------+------------------+---------+--------------------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+
|   1088|Dirty Dancing (1987)|  3.25002094679514|     1987|Drama|Musical|Rom...|  0.0|    1.0|     0.0|      0.0|  1.0|0.0|        0.0|    0.0|    0.0|    1.0|      0.0|      0.0|   0.0|    0.0|   0.0|     0.0|   0.0|   0.0|
|   1580|Men in Black (a.k...|3.5817083457378187|     1997|Action|Comedy|Sci-Fi|  0.0|    0.0|     0

*Checking null values in each column*

In [5]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+-----+----------+---------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+
|movieId|title|avg_rating|movieYear|genres|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|Horror|Western|Comedy|Children|Action|Sci-Fi|
+-------+-----+----------+---------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+
|      0|    0|         0|       24|     0|    0|      0|       0|        0|    0|  0|          0|      0|      0|      0|        0|        0|     0|      0|     0|       0|     0|     0|
+-------+-----+----------+---------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+



*Printing movies without year in it*

In [6]:
#Doing toPandas() for pretty printing
df.filter(col('movieYear').isNull()).select('title').toPandas()

,title
0,The King
1,Science Fiction Volume One: The Osiris Child
2,Mara
3,Angel Has Fallen
4,Trophy Kids
5,Falling Inn Love
6,The Perfect Date
7,In the Tall Grass
8,Tau
9,Ready Player One


In [7]:
df.describe().toPandas()

,summary,movieId,title,avg_rating,movieYear,genres,Crime,Romance,Thriller,Adventure,...,Mystery,Musical,Animation,Film-Noir,Horror,Western,Comedy,Children,Action,Sci-Fi
0,count,24009,24009,24009,23985,24009,24009,24009,24009,24009,...,24009,24009,24009,24009,24009,24009,24009,24009,24009,24009
1,mean,70788.93344162605,None,3.2003455170428925,1994.7507608922242,None,0.10745970261152069,0.14815277604231747,0.17372651922195842,0.09159065350493564,...,0.05747844558290641,0.0301137073597401,0.057436794535382564,0.010745970261152068,0.11041692698571369,0.019159481860968802,0.3237119413553251,0.055895705777000294,0.14290474405431297,0.07580490649339831
2,stddev,62574.92667477875,None,0.4939954796797395,22.038548812964635,None,0.30970326109458013,0.35525876163929826,0.37888203284832367,0.28845323933383626,...,0.23275938305937793,0.1709037405567116,0.23268017569029853,0.10310643614230844,0.3134152526066884,0.13708821565464424,0.4679013134320066,0.22972499630498883,0.3499829709256203,0.2646912177941029
3,min,1,"""""""Great Performances"""" Cats (1998)""",0.6785714285714286,1874,Action,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,max,208737,貞子3D (2012),4.483096085409253,2019,Western,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [8]:
col_names = df.columns
exc_cols = ['movieId', 'title', 'genres', 'movieYear']
feat_cols = list(set(col_names).difference(set(exc_cols)))
feat_cols

['Film-Noir',
 'Children',
 'Fantasy',
 'Sci-Fi',
 'Animation',
 'Mystery',
 'Western',
 'avg_rating',
 'Musical',
 'Drama',
 'War',
 'Action',
 'Comedy',
 'Documentary',
 'Horror',
 'Thriller',
 'Adventure',
 'Crime',
 'Romance']

### Creating features

### Vector assembler to create feature vector column

In [9]:
assembler = VectorAssembler(inputCols=feat_cols, outputCol='features', handleInvalid='skip')
final_df = assembler.transform(df)

In [10]:
final_df.show(5)

2022-05-20 00:43:38,211 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+------------------+---------+--------------------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+--------------------+
|movieId|               title|        avg_rating|movieYear|              genres|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|Horror|Western|Comedy|Children|Action|Sci-Fi|            features|
+-------+--------------------+------------------+---------+--------------------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+------+-------+------+--------+------+------+--------------------+
|   1088|Dirty Dancing (1987)|  3.25002094679514|     1987|Drama|Musical|Rom...|  0.0|    1.0|     0.0|      0.0|  1.0|0.0|        0.0|    0.0|    0.0|    1.0|      0.0|      0.0|   0.0|    0.0|   0.0|     0.0|   0.0|   0.0|(19,[7,8,9,18],[3...|
|   1580|Men in 

#### We might not need scaling, but did it anyway for learning purpose

In [11]:
# from pyspark.ml.feature import StandardScaler
# scaler = StandardScaler(inputCol='features',
#                        outputCol='scaled_feat',
#                        withStd=True,
#                        withMean=False
#                        )
# scaled_model = scaler.fit(final_df)
# cluster_df = scaled_model.transform(final_df)

#### Cluster Evaluator to check how close the points in individual clusters are ?

In [12]:
eval = ClusteringEvaluator(predictionCol='prediction',
                          featuresCol='features',
                          metricName='silhouette',
                          distanceMeasure='squaredEuclidean')

In [13]:
### KMeans clustering 

In [29]:
silhouette_score = []
print("""
Silhouette Scores for K Means Clustering
========================================
Model\tScore\t
=====\t=====\t
""")
start = time.time()
for k in range(10,1000,10):
    kmeans_algo = KMeans(featuresCol='features', k=k, seed=0, maxIter=1000, distanceMeasure="cosine")
    kmeans_fit = kmeans_algo.fit(final_df)
    output = kmeans_fit.transform(final_df)
    score = eval.evaluate(output)
    silhouette_score.append(score)
    # print(f"K{k}\t{np.round(score,3)}\t")
end = time.time()
print(f"Runtime: {end - start}")


Silhouette Scores for K Means Clustering
Model	Score	
=====	=====	



Runtime: 3172.1600263118744


In [30]:
silhouette_score

[0.2854618929032507,
 0.37609048185279786,
 0.41370275522265343,
 0.41157636370678335,
 0.41075226439528906,
 0.4214168620921757,
 0.4665414840167153,
 0.4575705686729076,
 0.4611476156843341,
 0.4593322097264788,
 0.4795405515914679,
 0.47004795165954566,
 0.4714257814412716,
 0.4988872771449631,
 0.4792137974634365,
 0.5132721624365166,
 0.49476625544833697,
 0.5207254512707246,
 0.5311711686587876,
 0.5355313819254457,
 0.5157645496525227,
 0.5437354905534553,
 0.5423925159092853,
 0.547279525829666,
 0.5482693668702856,
 0.5239182225779812,
 0.5432849415059935,
 0.5544288783314822,
 0.5542346498672988,
 0.5666550685072061,
 0.5599867885385463,
 0.5558631763565796,
 0.550071698259996,
 0.5609793572428325,
 0.5675082013911524,
 0.567316326422312,
 0.569285265092749,
 0.5778356070193694,
 0.5705188369798083,
 0.5678754352157689,
 0.5671040499015103,
 0.5753152743861127,
 0.5822252716028463,
 0.575947703263141,
 0.5886222886870929,
 0.5887403693657154,
 0.5872386988114976,
 0.580745177

In [31]:
import numpy as np
np.max(silhouette_score)

0.6376527608095093

#### The above score should give us idea if points inside clusters are close to each other.

In [15]:
spark.stop()